### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [9]:
import re

class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    regex = "[^a-zA-Z0-9^]"
    subst = " "
    # 정규식 적용
    for i in sequences :
      token = re.sub(regex, subst, i)
      # 소문자로 치환
      token = token.lower().split()
      result.append(token)
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    '''
    문제 1-2.
    '''
    tokens=self.preprocessing(sequences)
    
    #순서대로 어휘 사전 작성 위해 count 정의
    count=1
    
    for sent in tokens:   #문장 리스트에서 하나의 문장 가져오기
      for word in sent:   #하나의 문장에서 하나의 단어 가져오기
        if word not in self.word_dict.keys():   #어휘 사전에 단어가 없는 경우,
          self.word_dict[word] = count   #순서대로 어휘 등록
          count+=1  #등록된 다음 인덱스 번호를 더해줌
        else:
          pass
    self.fit_checker = True
  
  def transform(self, sequences):
    # 인덱싱 된 번호를 출력하기 위한 리스트 정의
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      '''
      문제 1-3.
      '''
      for sent in tokens:
        encode = []   #하나의 문장의 단어들을 인덱싱하여 저장하기 위한 리스트 정의
        for word in sent:
          if word in self.word_dict.keys():
            word = self.word_dict[word]   #정수 인덱싱
          else:
            word = self.word_dict['oov']   #oov의 값을 등록 안된 단어에 대입
            '''
            (참고용)아래의 코드를 사용하면 2개 이상의 등록되지 않은 단어도 다르게 사전에 등록하면서 사용가능.

            self.word_dict['oov'] = len(word_dict)+1  # oov의 값을 사전에 등록된 값보다 1 크게 유지 #fit 과정에서 사용
            self.word_dict[word] = self.word_dict['oov'] # 사전에 oov의 값으로 새로운 단어 등록
            self_word_dict[word] += 1 # 사전에 등록된 값보다 1 크게 유지
            '''
          encode.append(word)   #한 문장안의 단어들을 인덱싱하여 저장
        result.append(encode)   #인덱싱 된 리스트를 저장
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [10]:
sequences=['I go to school.', 'I LIKE pizza!','i go to room!','they have a pizza', 'i have to go!']
#sequences=['I go to school.', 'I LIKE pizza!']
tokenizer = Tokenizer()
processing = tokenizer.preprocessing(sequences)
print("전처리 과정 후 :",processing)
tokenizer.fit(sequences)
trans = tokenizer.transform(sequences)
print("정수 인덱싱 후 :" ,trans)
print("완성된 어휘 사전 :",tokenizer.word_dict)

전처리 과정 후 : [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza'], ['i', 'go', 'to', 'room'], ['they', 'have', 'a', 'pizza'], ['i', 'have', 'to', 'go']]
정수 인덱싱 후 : [[1, 2, 3, 4], [1, 5, 6], [1, 2, 3, 7], [8, 9, 10, 6], [1, 9, 3, 2]]
완성된 어휘 사전 : {'oov': 0, 'i': 1, 'go': 2, 'to': 3, 'school': 4, 'like': 5, 'pizza': 6, 'room': 7, 'they': 8, 'have': 9, 'a': 10}


### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [11]:
### 하나의 고유한 토큰당 IDF가 나와야함 고쳐야할 부분
from collections import Counter
from math import log
import pandas as pd

class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
    self.N = len(sequences) # 총 문장 개수 변수 지정
    self.vocab_IDF=[] #IDF 값 저장할 리스트 변수 지정

  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    '''
    문제 2-1.
    '''
    # 고유 token 값 구하기
    tokens = set(sum(tokenized, []))
    
    # IDF 구하기 (log(N/1+DF))
    # 토큰 1개당 전체 문장에 몇번 들어가있는지 확인하고 IDF값 도출
    # 오류발생 예제) 전체 문장이 4개 일때, 'i'라는 토큰이 3문장에서 발견되면 log(4/(3+1))으로 0으로 귀결되어 가중치 오류 발생
    for token in tokens:
      df = 0
      for sent in tokenized:   
        if sent.count(token) > 0 :
          df += 1
      self.vocab_IDF.append(log(self.N/(df+1)))
    
    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      # 고유 token 값 구하기
      tokens = list(set(sum(tokenized, [])))

      #TF-IDF 구하기 (tf * idf)
      result =[]
      for i in range(self.N):
        result.append([])
        sent = tokenized[i]
        for num in range(len(tokens)) :
          token = tokens[num]
          tf = sent.count(token)  #TF값 구하기
          idf = self.vocab_IDF[num]  #저장한 IDF값 불러오기
          result[-1].append(tf*idf)  #비어있는 []에 tf-idf값 저장

      self.tfidf_matrix = result

      return self.tfidf_matrix

    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

In [12]:
tfidf= TfidfVectorizer(tokenizer)

In [13]:
tfidf.fit(sequences)

In [14]:
t=tfidf.transform(sequences)

In [17]:
#IDF 값 확인하기
tfidf.vocab_IDF

[0.0,
 0.22314355131420976,
 0.22314355131420976,
 0.9162907318741551,
 0.9162907318741551,
 0.5108256237659907,
 0.9162907318741551,
 0.9162907318741551,
 0.5108256237659907,
 0.9162907318741551]

In [16]:
#TF-IDF 데이터프레임으로 확인하기
pd.DataFrame(t, index=sequences, columns=range(1,11))

,1,2,3,4,5,6,7,8,9,10
I go to school.,0.0,0.223144,0.223144,0.916291,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I LIKE pizza!,0.0,0.000000,0.000000,0.000000,0.916291,0.510826,0.000000,0.000000,0.000000,0.000000
i go to room!,0.0,0.223144,0.223144,0.000000,0.000000,0.000000,0.916291,0.000000,0.000000,0.000000
they have a pizza,0.0,0.000000,0.000000,0.000000,0.000000,0.510826,0.000000,0.916291,0.510826,0.916291
i have to go!,0.0,0.223144,0.223144,0.000000,0.000000,0.000000,0.000000,0.000000,0.510826,0.000000


In [18]:
#어휘 사전 확인
tokenizer.word_dict

{'a': 10,
 'go': 2,
 'have': 9,
 'i': 1,
 'like': 5,
 'oov': 0,
 'pizza': 6,
 'room': 7,
 'school': 4,
 'they': 8,
 'to': 3}